# Computer Vision - Mid Term  
**Name** - Tushar Choudhary  
**Roll no.** - 2019111019

## Question 1

From this snapshot of a video taken from the camera on a satelite can we:
1. Calibrate the camera
2. Find position of the satelite with respect to the earth at the instant this picture was taken

<img src="Images/map.jpg" alt="drawing" width="500"/>

**Why this question is interesting**  
I believe this question is interesting because I was able to calibrate this camera using only this image and some basic knowledge of geography. It was the first time I used the ECEF world coordinate system for a camera calibration setting. Usually the pictures taken for calibration are of a small scene where world coordinate points are easy to declare, but here I had to give a lot of thought on how I can locate the points in the world frame until I had thought of the ECEF frame. This was mainly because I had never seen such problem before. The final results gave me the intrensics of the camera and expected location of the the satelite.

**Answer**  
Given this image, the first thing I noticed are the brightly lit cities. I decided to choose some metro cities which were easy to spot and whose geography I was aware of. These cities were: 1) Mumbai, 2) Delhi, 3) Bangalore, 4) Hyderabad, 5) Chennai, 6) Kolkata. I spotted these cities and obtained the image points (in given order). I have marked them here in the image below:
<img src="Images/map_marked.png" alt="drawing" width="500"/>

Their pixel locations (in order) are:
```
x = [[437.1222392 321.40694845]
    [517.76339038 141.3344751 ]
    [525.59262835 436.49674663]
    [542.81695189 354.28974793]
    [574.91682757 437.27967043]
    [728.36989182 255.64134949]]
 ```

Now I needed to get the world points. I did some research and decided to use the ECEF world frame system to get the world points. This is a cartesian coordinate system with the origin located at the center of the earth. 
<img src="Images/ecef.png" alt="drawing" width="500"/>

However, I didn't know the ECEF coordinates for these cities directly. The only common info we have about a city's location generally are the `latitude`, `longitude`, and `height above sea level`. I hoped to get a relation between these two systems and successfully found a formula for conversion. I won't add the formula here as it isn't relative to the main problem but can be found at: https://danceswithcode.net/engineeringnotes/geodetic_to_ecef/geodetic_to_ecef.html. Hence, afterwards I searched for the latitude, longitude, and height above sea level for these cities and found the XYZ coordinates in ECEF system.

For save time I used the converter at: https://www.oc.nps.edu/oc2902w/coord/llhxyz.htm . The world points obtained in km were: 
```
X = [[1775.348 5762.779 2071.314]
     [1239.524 5463.165 3039.519]
     [1337.124 6071.808 1422.64]
     [1214.82  5966.6   1893.918]
     [1053.675 6124.097 1432.684]
     [ 168.239 5890.046 2433.091]]
```

Now the rest of the steps are straight forward. I applied DLT on the given 6 points. Let the projection matrix be P then we should have `x = PX`. Using elements of `x` and `X` I constructed a matrix `M` such that `Mp=0` where `p` is a column vector obtained by stacking elements of P. Then I applied SVD on M and got P using the last eigenvector. The projection matrix P obtained was:

```
P = [[-2.24028086e-02 -7.03737401e-02 -2.97957195e-02  5.34284937e+02]
     [-8.09202145e-03 -4.96260292e-02 -3.16483596e-02  3.85937067e+02]
     [-2.16897480e-05 -1.36021360e-04 -5.56640024e-05  1.00000000e+00]]
```

After that I decomposed P into K, R and T using RQ decomposition. Their values found were:

```
K = [[-7.42828602e+01  3.02812532e+00  5.30874538e+02]
     [ 1.39299884e-14  7.07094752e+01  3.93615691e+02]
     [-3.57043816e-13 -4.48618194e-13  1.00000000e+00]]

R = [[-0.98837588  0.15122972  0.01557833]
     [-0.04240019 -0.37260377  0.92702139]
     [ 0.14599773  0.91558506  0.37468475]]

T =  [1348.62534804 6384.09244589 1839.17284737]
```

To check the projection matrix P, I picked up Lucknow from the image and got its ECEF coordinates. On projecting the world point of Lucknow using the P obtained, the projected image point came quite close to the point manually selected from the image!

```
X_lko = [895.728, 5623.636, 2863.454]           // From ECEF frame
x_lko = [589.0094559208612, 171.08557939006]    // Obtained manually
proj  = [589.25374824 159.77095166 1]           // Obatined by projecting X_lko using P
```

As we see the projected point seems to lie quite close to the manually chosen point.     
We also found the position of the camera in the ECEF system as T = `[1348.62534804 6384.09244589 1839.17284737]` by the decomposition. The distance of the satelite from the earth's center is SQ_ROOT(1348.62534804<sup>2</sup> + 6384.09244589<sup>2</sup> + 1839.17284737<sup>2</sup>) = 6779.231774 km. Radius of earth near equator is approx 6378km, hence the satelite **seems to be flying about 400 km above the surface of the earth** which seems to be fair.

**The calculation invloving DLT was done using a code, I've attached it below:**

In [1]:
import numpy as np

def hom_coords(x,X):
    n = x.shape[0]
    size = (n,1)
    ones = np.ones(size)
    x = np.concatenate(
        (x,ones),
        axis = 1)
    X = np.concatenate(
        (X,ones),
        axis = 1)
    return x, X

def get_m(img_points, wld_points):
    x = img_points[:, 0]
    y = img_points[:, 1]
    X = wld_points[:, 0]
    Y = wld_points[:, 1]
    Z = wld_points[:, 2]
    n = img_points.shape[0] 
    a_x = np.c_[-1*X, -1*Y, -1*Z, -1*np.ones(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.multiply(x, X), np.multiply(x, Y), np.multiply(x, Z), x]
    a_y = np.c_[np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), -1*X, -1*Y, -1*Z, -1*np.ones(n), np.multiply(y, X), np.multiply(y, Y), np.multiply(y, Z), y]
    out = np.empty((2 * n, a_x.shape[1]))
    out[::2, :] = a_x
    out[1::2, :] = a_y
    return out

def DLT(x,X):
    m = get_m(x, X)
    U, S, V = np.linalg.svd(m)
    P = V[-1, :].reshape((3, 4))
    return P

def get_KRt(P):
    H = P[:, :3]
    h = P[:, -1]
    H_inv = np.linalg.pinv(H)
    t = - H_inv @ h
    R_tr, K_inv = np.linalg.qr(H_inv)
    R = R_tr.T
    K = np.linalg.pinv(K_inv)
    K = K / K[-1, -1]
    return K, R, t

In [2]:
x = [[437.1222392678604, 321.40694844869995],
 [517.7633903774433, 141.33447509720418],
 [525.5926283492474, 436.49674663422115],
 [542.8169518872166, 354.28974793027743],
 [574.9168275716136, 437.27967043140154],
 [728.3698918189752, 255.64134948554494]]

X = [
    [1775.348, 5762.779, 2071.314],
    [1239.524, 5463.165, 3039.519],
    [1337.124, 6071.808, 1422.64],
    [1214.82, 5966.6, 1893.918],
    [1053.675, 6124.097, 1432.684],
    [168.239, 5890.046, 2433.091]
]

x = np.array(x)
X = np.array(X)
x,X = hom_coords(x,X)
P = DLT(x, X)
K, R, t = get_KRt(P)
P = P/P[-1,-1]
print("P\n", P)
print('K\n', K)
print('R\n', R)
print('T:', t)

P
 [[-2.24028086e-02 -7.03737401e-02 -2.97957195e-02  5.34284937e+02]
 [-8.09202145e-03 -4.96260292e-02 -3.16483596e-02  3.85937067e+02]
 [-2.16897480e-05 -1.36021360e-04 -5.56640024e-05  1.00000000e+00]]
K
 [[-7.42828602e+01  3.02812532e+00  5.30874538e+02]
 [ 1.39299884e-14  7.07094752e+01  3.93615691e+02]
 [-3.57043816e-13 -4.48618194e-13  1.00000000e+00]]
R
 [[-0.98837588  0.15122972  0.01557833]
 [-0.04240019 -0.37260377  0.92702139]
 [ 0.14599773  0.91558506  0.37468475]]
T: [1348.62534804 6384.09244589 1839.17284737]


In [3]:
X_lko = [[895.728, 5623.636, 2863.454]]
x_lko = [[589.0094559208612, 171.08557939006]]
X_lko = np.array(X_lko)
x_lko = np.array(x_lko)
x_lko,X_lko = hom_coords(x_lko,X_lko)
proj = P@X_lko.T
proj = proj.T
proj = proj/proj[-1,-1]
print("Projected Lucknow = ", proj)

Projected Lucknow =  [[589.25374824 159.77095166   1.        ]]


## Question 2

Does the Harris corner detector's performance vary with respect to the angle of the corner? Judge the performance for some different angles based on the statistics you know.

**Why I found this problem interesting**  
In the lecture, we only compared eigenvalues for cases of plane, edge and corner, but never discussed that how the performance shall vary for different angles of a corner. Will there be any difference at all? If so then for which angles will Harris corner detector perform the best? Will θ and 2π-θ give the same score? These I some possible questions that could have been explored in the lecture but weren't. So I will test some cases on my own and note down what results I get.

**Answer**

The pseudo-code for analyzing the results would be, taking a patch I containing a corner and convolving it with the `d/dx = [-1 0 1]` and `d/dy = [-1 0 1]'` filters to obtain the I<sub>X</sub> and I<sub>Y</sub> matrices respectively. I<sub>X</sub> and I<sub>Y</sub> here stand for 𝛿I/𝛿X and 𝛿I/𝛿Y. Using these matrices we will get the H matrix.  
The H matrix will be equal to $\sum_{(x,y) ∈ patch}$ of: 
$$
\left(\begin{array}{cc} 
Ix * Ix & Ix * Iy\\
Ix * Iy & Iy * Iy
\end{array}\right)
$$ 
Finally we will analyse the following stats:
1. Eigen Values λ1 and λ2
2. Det(H)/Trace(H)
3. Harris Response value

In Harris Response value I'll use K = 0.05. Hence its final value will be Det(H) - 0.05 * trace(H)<sup>2</sup>.

**Theta = 0°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 0 & 0 & \\
& 0 & 0 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  0   
Trace(H) =  0   
Can't get Det(H)/Trace(H) as Trace(H)=0.  
Harris response =  0   
L1 and L2 = 0 and 0 respectively

**Theta = 45°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & -25 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & -25 & -25 & \\
& 0 & 0 & 0 & 25 & 25 & -25 & -25 & 0 & \\
& 0 & 0 & 0 & 25 & 0 & -25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & -25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & -25 & -25 & \\
& 0 & 0 & 0 & 0 & 0 & -25 & -25 & 0 & \\
& 0 & 0 & 0 & 0 & -25 & -25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 8125 & 3750 & \\
& 3750 & 5000 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  26562500   
Trace(H) =  13125   
Det(H)/Trace(H) =  2023   
Harris response =  17949218   
L1 and L2 = 10625 and 2500 respectively.

**Theta = 90°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & -25 & -25 & -25 & -25 & \\
& 0 & 0 & 0 & 0 & -25 & -25 & -25 & -25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 5000 & -625 & \\
& -625 & 5000 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  24609374   
Trace(H) =  10000   
Det(H)/Trace(H) =  2460   
Harris response =  19609374   
L1 and L2 = 5625 and 4375 respectively.

**Theta = 135°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 25 & 25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 25 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & -25 & -25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & -25 & -25 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & -25 & -25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & -25 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 9375 & -5000 & \\
& -5000 & 5000 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  21875000   
Trace(H) =  14375   
Det(H)/Trace(H) =  1521   
Harris response =  11542968   
L1 and L2 = 12645 and 1729 respectively.

**Theta = 180°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & 25 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 25 & 25 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 10000 & 0 & \\
& 0 & 0 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  0   
Trace(H) =  10000   
Det(H)/Trace(H) =  0   
Harris response =  -5000000   
L1 and L2 = 10000 and 0 respectively.

**Theta = 225°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & -25 & -25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & -25 & -25 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & -25 & -25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & -25 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 25 & 25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 25 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 9375 & -5000 & \\
& -5000 & 5000 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  21875000   
Trace(H) =  14375   
Det(H)/Trace(H) =  1521   
Harris response =  11542968   
L1 and L2 = 12645 and 1729 respectively.

**Theta = 270°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 25 & 25 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 5000 & -625 & \\
& -625 & 5000 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  24609374   
Trace(H) =  10000   
Det(H)/Trace(H) =  2460   
Harris response =  19609374   
L1 and L2 = 5625 and 4375 respectively.

**Theta = 315°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 0 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 0 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 0 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 0 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 0 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & -25 & -25 & 0 & 0 & 25 & \\
& 0 & 0 & 0 & -25 & -25 & 0 & 25 & 25 & \\
& 0 & 0 & 0 & -25 & -25 & 25 & 25 & 0 & \\
& 0 & 0 & 0 & -25 & 0 & 25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 25 & 25 & \\
& 0 & 0 & 0 & 0 & 0 & 25 & 25 & 0 & \\
& 0 & 0 & 0 & 0 & 25 & 25 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 25 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 8125 & 3750 & \\
& 3750 & 5000 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  26562500   
Trace(H) =  13125   
Det(H)/Trace(H) =  2023   
Harris response =  17949218   
L1 and L2 = 10625 and 2500 respectively.

**Theta = 360°**  
  
Image patch I = 
$$
\left(\begin{array}{cc}
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
& 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & 25 & \\
\end{array}\right)
$$ 

After convolving I with d/dx we get Ix = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
After convolving I with d/dy we get Iy = 
$$
\left(\begin{array}{cc}
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
& 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
\end{array}\right)
$$  
  
Using Ix and Iy we get H = 
$$
\left(\begin{array}{cc}
& 0 & 0 & \\
& 0 & 0 & \\
\end{array}\right)
$$  

From this H we get:  
Det(H) =  0   
Trace(H) =  0   
Can't get Det(H)/Trace(H) as Trace(H)=0.  
Harris response =  0   
L1 and L2 = 0 and 0 respectively.

**Summary:**  

|Angle |λ1    |λ2     |Det(H)     |Trace(H)|Det(H)/Trace(H)|Harris response value   |
|------|------|-------|-----------|--------|-------|----------|
|0     |0     |  0    |  0        |  0     |  -    |  0
|45    |10625 |  2500 |  26562500 |  13125 |  2023 |  17949218
|90    |5625  |  4375 |  24609374 |  10000 |  2460 |  19609374
|135   |12645 |  1729 |  21875000 |  14375 |  1521 |  11542968
|180   |10000 |  0    |  0        |  10000 |  0    |  -5000000
|225   |12645 |  1729 |  21875000 |  14375 |  1521 |  11542968
|270   |5625  |  4375 |  24609374 |  10000 |  2460 |  19609374
|315   |10625 |  2500 |  26562500 |  13125 |  2023 |  17949218
|360   |0     |  0    |  0        |  0     |  -    |  0

**Observations**
* The highest score was obtained on θ=90 and θ=270. The corner detect seems to be most well suited to these angles.
* The Harris corner detector doesn't give seem performance on angles symmetric about 90. That is θ and 180-θ have different performance on the Harris corner detector. The detector gives higher scores for θ=45 compared to θ=135.
* The detector gives exactly same performace on angles symmetric about 180. That is θ and 360-θ have the same performace as they both get the same scores.
* For edges, the Harris response value was found to be negative which we know to be a true fact.

## Question 3

Suppose you have to develop an app to apply a face filter on live feed from phone's selfie camera, somthing like instagram filters. Which face detection algorithm will you use? Why (explain in detail)?

**Why I found this problem interesting**

I think this problem is interesting because its an application of computer vision that many people have been using in their daily life since quite some time now. Tech companies like Facebook actively work on this. Someone in the tech industry who does not actively research on but wants to get into computer vision might face such tasks at his work. These days most of such functions are available through libraries but its still imporant to know such algorithms which run in real time and give accurate and efficient results, specially if your works revolves around it.

**Answer**  
Since the application shows face filter from live camera feed, we need an algorithm that will perform face detection in real time with good accuracy. I'll use the Voila-Jones algorithm for face detection. It has low inference times and good accuracy. It is so efficient that many phone face unlock systems use this algorithm. Apple was using this till 2015 and Snapchat still uses this algorithm till today.

Generally in any face detection algorithm, to detect face from a given image, we use windows of different scales and pass them over the image. For each window location, we calculate a set of feature vectors and compare this set of feature vectors with other feature vectors which have a face in it and determine if the window position currently contains a face. To consider all possible scales, for an image of size 1024x1024, we usually use windows of sizes starting from 24x24 uptil 1024x1024, with approximately a 1.25 times scale change between successive window sizes. This gives about 17 filter sizes in total. Evaluating all these windows over the image at different positions requires calculating different features at all possible locations. This makes the task computationally expensive.

Voila and Jones introduced the concept of integral images which made the task of calculating features for a window significantly faster. Integral images can be generated from the original image where each pixel has a value equal to the sum of the values of all pixels above it and to its left. This calculation of integral image is only a one time task, after we have calculated it we can increase speed of feature extraction as follows:  
S(i,j,x,y) = S(0,0,x,y) - S(0,0,i-1,y) - S(0,0,x,j-1) + S(0,0,i-1,j-1)  
Now each using this formula, we can calculate the some of values on O(1) which increases the speed during inference.

<img src="Images/3.jpg" alt="drawing" width="500"/>

Another important thing that Voila and Jones observed was that generally there are about 10 to 15 windows that contain faces in majority cases so it would be ineffecient to evaluate all features for all windows. Instead the features should be evaluated in such an order so that features which have high true negative rate should be evaluated first and then features with increasingly higher true positive rate should be evaluated. Using this method we reduce the rate of false positive and greatly increase the accuracy. This speeds up the algorithms greatly as not much time is spent on windows which dont have face in them.   
Hence the algorithms proposed by Vila and Jones helps us get a realtime face detection algorithm.